In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
# os.environ['RAY_DEDUP_LOGS'] = '0'
# os.environ['NCCL_DEBUG'] = 'WARN'
# os.environ['RAY_ENABLE_RECORD_ACTOR_TASK_LOGGING'] = '1'
# export RAY_BACKEND_LOG_LEVEL=debug
import functools as ft
import time

import numpy as np
import ray
import torch
import tqdm.auto
import tqdm.notebook
tqdm.notebook.tqdm = tqdm.auto.tqdm  # Enable text output in notebooks

import celltrip

# Detect Cython
CYTHON_ACTIVE = os.path.splitext(celltrip.utility.general.__file__)[1] in ('.c', '.so')
print(f'Cython is{" not" if not CYTHON_ACTIVE else ""} active')


Cython is active


- High priority
  - Auto set max GPUs for update (?)
  - Implement stages
  - Add checkpoints
  - Add model loading
  - Add train/val to dataloader
  - Partition detection in `train_policy`
  - Script arguments, including address for ray
- Medium priority
  - Eliminate passing of persistent storage for memory objects
  - Add hook for wandb, etc.
  - Add state manager to env and then parallelize in analysis, maybe make `analyze` function
- Low priority
  - Seed policy initialization and unseed update, add reproducibility tag to wait for all rollouts before updating
  - Verify worker timeout
  - Subtract working memory on host node
  - Local data loading per worker

In [3]:
# # Arguments
# import argparse
# parser = argparse.ArgumentParser(description='Train CellTRIP model', formatter_class=argparse.ArgumentDefaultsHelpFormatter)

# # TODO: Figure out how to format arguments for appending h5ad files
# parser.add_argument('datasets', type=str, required=False, help='.h5ad files to use for data')
# parser.add_argument('--concatenate', type=str, required=False, help=
#     '.h5ad files to concatenate as a single modality, may be used multiple times')

# group = parser.add_argument_group('General')
# group.add_argument('--seed', default=42, type=int, help='**Seed for random calls during training')


In [ ]:
# Start timer
start_time = time.perf_counter()

In [4]:
# Read data
fnames = ['../data/MERFISH/expression.h5ad', '../data/MERFISH/spatial.h5ad']
partition_cols = None
adatas = celltrip.utility.processing.read_adatas(*fnames, on_disk=False)
celltrip.utility.processing.test_adatas(*adatas, partition_cols=partition_cols)

# Construct dataloader
dataloader = celltrip.utility.processing.PreprocessFromAnnData(
    *adatas, partition_cols=partition_cols, num_nodes=200, pca_dim=128, seed=42)
modalities, adata_obs, adata_vars = dataloader.sample()

# Initialize Ray
ray.shutdown()
# ray.init(
#     resources={'VRAM': torch.cuda.get_device_properties(0).total_memory},
#     dashboard_host='0.0.0.0')
ray.init(
    address='ray://127.0.0.1:10001',
    runtime_env={
        'env_vars': {
            # NOTE: Important, NCCL will timeout if network device is non-standard
            'NCCL_SOCKET_IFNAME': 'tailscale',
            # 'NCCL_DEBUG': 'WARN',
            'RAY_DEDUP_LOGS': '0',
        }})

# Initialize distributed manager
policy_init, memory_init = celltrip.train.get_train_initializers(
    3, [m.shape[1] for m in modalities])
distributed_manager = celltrip.train.DistributedManager(
    # modalities=modalities, env_init=env_init,
    policy_init=policy_init,
    memory_init=memory_init)

# Available resources
import json
print(json.dumps(ray.available_resources(), indent=2, sort_keys=False))

# Perform training
celltrip.train.train_policy(distributed_manager, dataloader)

/tmp/ipykernel_12045/1487911653.py:8: RuntimeWarning: Modality 1 too small for PCA (2 features), skipping
  dataloader = celltrip.utility.processing.PreprocessFromAnnData(
2025-03-20 02:53:18,947	INFO client_builder.py:244 -- Passing the following kwargs to ray.init() on the server: log_to_driver
SIGTERM handler is not set because current thread is not the main thread.


{
  "node:100.64.246.20": 1.0,
  "accelerator_type:G": 1.0,
  "node:__internal_head__": 1.0,
  "node:100.85.187.118": 1.0,
  "CPU": 48.0,
  "object_store_memory": 62368414924.0,
  "accelerator_type:RTX": 1.0,
  "VRAM": 57291112448.0,
  "GPU": 2.0,
  "memory": 144151812507.0
}


(NCCLUniqueIDStore pid=14383) The NCCL ID has not been set yet for store 8dc42047ef47a89d46bb4a68f46182df4cae1779.


In [ ]:
# End timer
time.perf_counter() - start_time

In [ ]:
# # Cancel
# # dm.cancel()
# # dm.clean()
# # dm.rollout(dummy=True)
# # dm.wait()

# # Clear locks
# distributed_manager.policy_manager.release_locks.remote()

# # Get policy
# device = 'cuda'
# policy = policy_init().to(device)
# celltrip.train.set_policy_state(policy, ray.get(distributed_manager.policy_manager.get_policy_state.remote()))

# # Get memory
# memory = memory_init(policy)
# memory.append_memory(
#     *ray.get(distributed_manager.policy_manager.get_memory_storage.remote()))

# # Update remote policy
# distributed_manager.policy_manager.release_locks.remote()
# ray.get(distributed_manager.update())

# # Get state of job from ObjectRef
# import ray.util.state
# object_id = dm.futures['simulation'][0].hex()
# object_state = ray.util.state.get_objects(object_id)[0]
# object_state.task_status
